In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [2]:
dantri_url = 'https://dantri.com.vn/'
response = requests.get(dantri_url)
response.text

'<!DOCTYPE html>\r\n<html lang="vi" xmlns="http://www.w3.org/1999/xhtml">\r\n<head><meta charset="UTF-8"><meta name="viewport" content="width=device-width, initial-scale=1"><meta name="format-detection" content="telephone=no"><meta name="robots" content="index,follow,all,max-image-preview:large"><title>Tin tức Việt Nam và quốc tế nóng, nhanh, cập nhật 24h | Báo Dân trí</title><meta name="description" content="Đọc báo dantri - Tin tức mới nhất, Thông tin nhanh chính xác được cập nhật hàng giờ. báo nói đọc tin tức online Việt Nam Thế giới nóng nhất trong ngày, Kinh doanh Việc làm, Pháp luật Đời sống, Giáo dục Sức khỏe, Thể thao Giải trí, Công nghệ, Văn hóa Xã hội, Ôtô Xe máy, Tình yêu, Nhân ái, Bạn đọc"><meta name="keywords" content="dantri,báo điện tử dantri,thời tiết,tin tức,tin nhanh,thể thao,dự báo thời tiết,tin the thao,thể thao 24h,giá vàng hôm nay,tin the gioi,đọc báo,tin moi,giá vàng,thời tiết hôm nay,tin tức online,bóng đá việt nam,pháp luật,dự báo thời tiết hôm nay,tin tức 24 g

In [3]:
soup = BeautifulSoup(response.text, 'html.parser')
soup

<!DOCTYPE html>

<html lang="vi" xmlns="http://www.w3.org/1999/xhtml">
<head><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1" name="viewport"/><meta content="telephone=no" name="format-detection"/><meta content="index,follow,all,max-image-preview:large" name="robots"/><title>Tin tức Việt Nam và quốc tế nóng, nhanh, cập nhật 24h | Báo Dân trí</title><meta content="Đọc báo dantri - Tin tức mới nhất, Thông tin nhanh chính xác được cập nhật hàng giờ. báo nói đọc tin tức online Việt Nam Thế giới nóng nhất trong ngày, Kinh doanh Việc làm, Pháp luật Đời sống, Giáo dục Sức khỏe, Thể thao Giải trí, Công nghệ, Văn hóa Xã hội, Ôtô Xe máy, Tình yêu, Nhân ái, Bạn đọc" name="description"/><meta content="dantri,báo điện tử dantri,thời tiết,tin tức,tin nhanh,thể thao,dự báo thời tiết,tin the thao,thể thao 24h,giá vàng hôm nay,tin the gioi,đọc báo,tin moi,giá vàng,thời tiết hôm nay,tin tức online,bóng đá việt nam,pháp luật,dự báo thời tiết hôm nay,tin tức 24 giờ,tin nong,thời 

## Tạo danh sách các links để truy cập
- Ý tưởng là duyệt qua để tìm url của các category
- Sau đó vào từng category và duyệt qua từng page
- crawl link bài viết trên từng page đó để tạo thành list các link phục vụ cho việc cào nội dung

In [4]:
# Lấy url của các thể loại báo

def get_url_of_each_category(soup):
    url_categories = soup.find('ol', {'class': 'nf-menu'}).find_all('a', href=True)

    list_url_category = []
    list_name_category = []

    name_categories = [
        'Thế giới', 'Kinh doanh', 'Bất động sản', 'Thể thao', 'Việc làm',
        'Nhân ái', 'Sức khỏe', 'Giải trí', 'Xe ++', 'Giáo dục', 
        'An sinh', 'Pháp luật', 'Du lịch', 'Đời sống', 'Tình yêu']

    for a in url_categories:
        if a.text in name_categories:
            list_url_category.append(a['href'])
            list_name_category.append(a.text)
            name_categories.remove(a.text)

    return list_url_category, list_name_category

In [5]:
list_url_category, list_name_category = get_url_of_each_category(soup)
for name, url in zip(list_name_category, list_url_category):
    print(f"Category: {name} - {url}")

Category: Kinh doanh - https://dantri.com.vn/kinh-doanh.htm
Category: Sức khỏe - https://dantri.com.vn/suc-khoe.htm
Category: Bất động sản - https://dantri.com.vn/bat-dong-san.htm
Category: Đời sống - https://dantri.com.vn/doi-song.htm
Category: Thể thao - https://dantri.com.vn/the-thao.htm
Category: Giải trí - https://dantri.com.vn/giai-tri.htm
Category: Nhân ái - https://dantri.com.vn/tam-long-nhan-ai.htm
Category: Thế giới - https://dantri.com.vn/the-gioi.htm
Category: Du lịch - https://dantri.com.vn/du-lich.htm
Category: Xe ++ - https://dantri.com.vn/o-to-xe-may.htm
Category: Giáo dục - https://dantri.com.vn/giao-duc.htm
Category: Việc làm - https://dantri.com.vn/lao-dong-viec-lam.htm
Category: An sinh - https://dantri.com.vn/lao-dong-viec-lam/an-sinh.htm
Category: Pháp luật - https://dantri.com.vn/phap-luat.htm
Category: Tình yêu - https://dantri.com.vn/tinh-yeu-gioi-tinh.htm


In [6]:
# Lấy danh sách các url page của mỗi thể loại

def get_list_page_for_each_category(url_categories: list) -> list:
  for i in range(len(url_categories)):
    url_categories[i] = url_categories[i].replace(".htm","")
    
  list_pages_for_each_category = []
  for category in url_categories:
    for page in range(1, 25): # lớn hơn vẫn được nhưng vì dataset lấy khoảng 5000 mẫu thôi 
      url_page = f'{category}/trang-{page}.htm'
      list_pages_for_each_category.append(url_page)
  
  return list_pages_for_each_category

In [7]:
list_pages_for_each_category = get_list_page_for_each_category(list_url_category)
list_pages_for_each_category

['https://dantri.com.vn/kinh-doanh/trang-1.htm',
 'https://dantri.com.vn/kinh-doanh/trang-2.htm',
 'https://dantri.com.vn/kinh-doanh/trang-3.htm',
 'https://dantri.com.vn/kinh-doanh/trang-4.htm',
 'https://dantri.com.vn/kinh-doanh/trang-5.htm',
 'https://dantri.com.vn/kinh-doanh/trang-6.htm',
 'https://dantri.com.vn/kinh-doanh/trang-7.htm',
 'https://dantri.com.vn/kinh-doanh/trang-8.htm',
 'https://dantri.com.vn/kinh-doanh/trang-9.htm',
 'https://dantri.com.vn/kinh-doanh/trang-10.htm',
 'https://dantri.com.vn/kinh-doanh/trang-11.htm',
 'https://dantri.com.vn/kinh-doanh/trang-12.htm',
 'https://dantri.com.vn/kinh-doanh/trang-13.htm',
 'https://dantri.com.vn/kinh-doanh/trang-14.htm',
 'https://dantri.com.vn/kinh-doanh/trang-15.htm',
 'https://dantri.com.vn/kinh-doanh/trang-16.htm',
 'https://dantri.com.vn/kinh-doanh/trang-17.htm',
 'https://dantri.com.vn/kinh-doanh/trang-18.htm',
 'https://dantri.com.vn/kinh-doanh/trang-19.htm',
 'https://dantri.com.vn/kinh-doanh/trang-20.htm',
 'https:/

In [8]:
len(list_pages_for_each_category)

360

In [9]:
# Lấy danh sách các url bài viết trên mỗi page
def get_articles_url_on_each_page(url):
  try:
    list_url_articles = []

    Content = requests.get(url).content
    Soup = BeautifulSoup(Content, 'html.parser')
    url_articles = Soup.find('div', {'class': 'article list'}).find_all('a', href=True)

    for a in url_articles:
      list_url_articles.append(a['href'])
    return list_url_articles
  
  except Exception as e:
    print(f"[ERROR] Không thể cào trang: {url}: {e}")
    return []

In [10]:
list_url_category, list_name_category = get_url_of_each_category(soup)

seen_urls = set()
all_articles = []

for name, url in zip(list_name_category, list_url_category):
    print(f"Đang cào category: {name} - {url}")
    list_pages_for_each_category = get_list_page_for_each_category([url])
    for page in list_pages_for_each_category:
        print(f"---Đang cào page: {page}")
        articles = get_articles_url_on_each_page(page)
        for link in articles:
            if link not in seen_urls:
                all_articles.append((name, link))
                seen_urls.add(link)

print(f"Tổng số bài đã cào: {len(all_articles)}")

Đang cào category: Kinh doanh - https://dantri.com.vn/kinh-doanh.htm
---Đang cào page: https://dantri.com.vn/kinh-doanh/trang-1.htm
---Đang cào page: https://dantri.com.vn/kinh-doanh/trang-2.htm
---Đang cào page: https://dantri.com.vn/kinh-doanh/trang-3.htm
---Đang cào page: https://dantri.com.vn/kinh-doanh/trang-4.htm
---Đang cào page: https://dantri.com.vn/kinh-doanh/trang-5.htm
---Đang cào page: https://dantri.com.vn/kinh-doanh/trang-6.htm
---Đang cào page: https://dantri.com.vn/kinh-doanh/trang-7.htm
---Đang cào page: https://dantri.com.vn/kinh-doanh/trang-8.htm
---Đang cào page: https://dantri.com.vn/kinh-doanh/trang-9.htm
---Đang cào page: https://dantri.com.vn/kinh-doanh/trang-10.htm
---Đang cào page: https://dantri.com.vn/kinh-doanh/trang-11.htm
---Đang cào page: https://dantri.com.vn/kinh-doanh/trang-12.htm
---Đang cào page: https://dantri.com.vn/kinh-doanh/trang-13.htm
---Đang cào page: https://dantri.com.vn/kinh-doanh/trang-14.htm
---Đang cào page: https://dantri.com.vn/kinh

In [11]:
all_articles

[('Kinh doanh',
  'https://dantri.com.vn/kinh-doanh/gia-han-nhan-ho-so-vong-1-vietnam-esg-awards-2025-20251004095137197.htm'),
 ('Kinh doanh',
  'https://dantri.com.vn/kinh-doanh/gia-vang-tien-sat-moc-139-trieu-dongluong-20251004082107763.htm'),
 ('Kinh doanh',
  'https://dantri.com.vn/kinh-doanh/visa-k-trung-quoc-mo-rong-cua-thu-hut-nhan-tai-the-gioi-20251003220453495.htm'),
 ('Kinh doanh',
  'https://dantri.com.vn/kinh-doanh/hoang-huong-bi-bat-chuyen-gia-chi-ro-thu-doan-tron-thue-qua-ho-kinh-doanh-20251003224737030.htm'),
 ('Kinh doanh',
  'https://dantri.com.vn/kinh-doanh/hoang-huong-bi-bat-he-lo-de-che-kinh-doanh-va-chieu-tro-quang-cao-no-20251003210628410.htm'),
 ('Kinh doanh',
  'https://dantri.com.vn/kinh-doanh/tao-ra-51-gdp-khoi-tu-nhan-khat-khao-duoc-trao-quyen-duoc-cong-nhan-20251003095426584.htm'),
 ('Kinh doanh',
  'https://dantri.com.vn/kinh-doanh/nganh-xi-mang-la-loi-giai-bat-ngo-cho-70000-tan-rac-thaingay-20251003174942514.htm'),
 ('Kinh doanh',
  'https://dantri.com.vn/

In [12]:
all_articles_df = pd.DataFrame(all_articles, columns=['category', 'url'])
all_articles_df

,category,url
0,Kinh doanh,https://dantri.com.vn/kinh-doanh/gia-han-nhan-...
1,Kinh doanh,https://dantri.com.vn/kinh-doanh/gia-vang-tien...
2,Kinh doanh,https://dantri.com.vn/kinh-doanh/visa-k-trung-...
3,Kinh doanh,https://dantri.com.vn/kinh-doanh/hoang-huong-b...
4,Kinh doanh,https://dantri.com.vn/kinh-doanh/hoang-huong-b...
...,...,...
6453,Tình yêu,https://dantri.com.vn/tinh-yeu-gioi-tinh/lan-d...
6454,Tình yêu,https://dantri.com.vn/tinh-yeu-gioi-tinh/moi-l...
6455,Tình yêu,https://dantri.com.vn/tinh-yeu-gioi-tinh/nho-m...
6456,Tình yêu,https://dantri.com.vn/tinh-yeu-gioi-tinh/toi-c...


# Vào từng bài viết và lấy các thông tin cần thiết

- Phần này khá đơn giản do các thông tin này được load ngay khi truy cập vào trang.
- Sử dụng BS4 là đủ để crawl

In [13]:
def get_article_detail(url):
    try:
        resp = requests.get(url, timeout=10)
        resp.encoding = "utf-8"
        soup = BeautifulSoup(resp.text, "html.parser")

        # Tiêu đề
        title_tag = soup.find("h1", class_="title-page")
        title = title_tag.get_text(strip=True) if title_tag else None

        # Sapo (đoan mở đầu)
        sapo_tag = soup.find("h2", class_="singular-sapo")
        sapo = sapo_tag.get_text(strip=True) if sapo_tag else None

        # Nội dung
        content_tag = soup.find("div", class_="singular-content")
        if content_tag:
            content = " ".join([p.get_text(strip=True) for p in content_tag.find_all("p")])
        else:
            content = None

        # Tác giả
        author_tag = soup.find("div", class_="author-name")
        author = author_tag.get_text(strip=True) if author_tag else None

        # Link của tác giả
        author_link = author_tag.a["href"] if author_tag and author_tag.a else None

        # Main section và Sub section
        # Tìm breadcrumb
        breadcrumb = soup.find("ul", class_="dt-text-c808080")

        links = breadcrumb.find_all("a")
        main_section = links[0].get_text(strip=True) if breadcrumb else None
        sub_section = links[1].get_text(strip=True) if (breadcrumb and len(breadcrumb) > 1) else None

        # Ngày đăng
        date_tag = soup.find("time", class_="author-time")
        post_date = date_tag.get_text(strip=True) if date_tag else None

        return {
            "link": url,
            "title": title,
            "sapo": sapo,
            "content": content,
            "author_name": author,
            "author_link": author_link,
            "main_section": main_section,
            "sub_section": sub_section,
            "post_date": post_date,
        }

    except Exception as e:
        # Nếu có bất kỳ lỗi nào (thường là trang không đúng định dạng chung)
        # In ra lỗi và trả về dict với toàn bộ trường = None
        print(f"Lỗi khi lấy {url}: {e}")
        return {
            "link": url,
            "title": None,
            "sapo": None,
            "content": None,
            "author_name": None,
            "author_link": None,
            "main_section": None,
            "sub_section": None,
            "post_date": None,
        }

In [14]:
# Trang bình thường
url = "https://dantri.com.vn/the-thao/kinh-ngu-dau-tien-cua-viet-nam-gianh-2-hcv-o-giai-chau-a-20251001232131492.htm"
data = get_article_detail(url)
data


{'link': 'https://dantri.com.vn/the-thao/kinh-ngu-dau-tien-cua-viet-nam-gianh-2-hcv-o-giai-chau-a-20251001232131492.htm',
 'title': 'Kình ngư đầu tiên của Việt Nam giành 2 HCV ở giải châu Á',
 'sapo': '(Dân trí) - Vận động viên (VĐV) bơi Nguyễn Huy Hoàng đã đi vào lịch sử khi trở thành kình ngư đầu tiên của Việt Nam giành 2 Huy chương vàng (HCV) ở giải châu Á khi về nhất nội dung 800m tự do nam vào tối 1/10.',
 'content': 'Kình ngư số một Việt Nam Nguyễn Huy Hoàng tiếp tục tỏa sáng rực rỡ tại giải vô địchthể thaodưới nước châu Á 2025. Tối nay (1/10), anh đã giành thêm HCV nội dung 800m tự do nam với thành tích 7 phút 57 giây 58. Huy Hoàng giành Huy chương vàng ở nội dung bơi 800m tại giải vô địch thể thao dưới nước châu Á 2025 (Ảnh chụp màn hình). Ở phần thi này, Huy Hoàng phải đối đầu với nhiều đối thủ mạnh như Rawat Kushagra (Ấn Độ), Khiew Hoe Yean (Malaysia), Xu Haibo (Trung Quốc), Sibirtsev Ilya (Uzbekistan) hay Tanaka Shun (Nhật Bản). Tuy nhiên, kình ngư quê Quảng Bình nhanh chóng

In [15]:
# Trang khác định dạng chung
invalid_url = "https://dantri.com.vn/bat-dong-san/hang-trieu-gia-dinh-an-cu-nho-quy-nha-o-xa-hoi-nguoi-viet-mong-som-co-nha-20251002125623821.htm"
invalid_data = get_article_detail(invalid_url)
invalid_data

Lỗi khi lấy https://dantri.com.vn/bat-dong-san/hang-trieu-gia-dinh-an-cu-nho-quy-nha-o-xa-hoi-nguoi-viet-mong-som-co-nha-20251002125623821.htm: 'NoneType' object has no attribute 'find_all'


{'link': 'https://dantri.com.vn/bat-dong-san/hang-trieu-gia-dinh-an-cu-nho-quy-nha-o-xa-hoi-nguoi-viet-mong-som-co-nha-20251002125623821.htm',
 'title': None,
 'sapo': None,
 'content': None,
 'author_name': None,
 'author_link': None,
 'main_section': None,
 'sub_section': None,
 'post_date': None}

# Cào lượng reactions và chi tiết reactions

- Do reactions thuộc dạng lazyload nên BS4 bình thường không thể đọc được.
- Cần dùng selenium để giả lập việc mở trang web, sau đó lăn chuột xuống giữa trang vì reactions
nằm ở giữa trang và cần khoảng 1-2s để load
- Nếu muốn lấy thêm data nằm ở cuối trang thì cần thêm việc lăn chuột tới cuối trang nữa
- Dantri là 1 trang báo lazyload khá nhiều phần

In [16]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

In [17]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

def get_reactions(data: dict, url: str) -> dict:
    try:
        if data is None:
            data = {}

        driver = webdriver.Chrome()
        driver.get(url)

        # Scroll chậm trên toàn trang
        scroll_height = driver.execute_script("return document.body.scrollHeight")
        step = scroll_height // 10   # chia trang thành 10 phần
        for i in range(0, scroll_height, step):
            driver.execute_script(f"window.scrollTo(0, {i});")
            time.sleep(1.5)  # đợi load JS
        # đảm bảo tới cuối
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        html = driver.page_source
        driver.quit()

        soup = BeautifulSoup(html, "html.parser")
        buttons = soup.find_all("button", {"aria-label": True})

        # Chỉ giữ 4 loại react cần thiết
        labels = {"Bổ ích", "Xúc động", "Cảm hứng", "Độc đáo"}

        total_reacts = 0
        detailed_reacts = {}

        for btn in buttons:
            label = btn.get("aria-label", "")
            if label not in labels:
                continue

            count_text = btn.get_text(strip=True)
            try:
                count = int(count_text) if count_text.isdigit() else 0
            except:
                count = 0

            detailed_reacts[label] = count
            total_reacts += count

        data["total_reacts"] = total_reacts
        data["detailed_reacts"] = detailed_reacts

        return data

    except Exception as e:
        print(f"Lỗi khi lấy reactions của links: {url}: {e}")
        data["total_reacts"] = 0
        data["detailed_reacts"] = None
        return data



In [18]:
url = "https://dantri.com.vn/the-thao/kinh-ngu-dau-tien-cua-viet-nam-gianh-2-hcv-o-giai-chau-a-20251001232131492.htm"
data = get_article_detail(url)
test_react = get_reactions(data, url)
test_react

{'link': 'https://dantri.com.vn/the-thao/kinh-ngu-dau-tien-cua-viet-nam-gianh-2-hcv-o-giai-chau-a-20251001232131492.htm',
 'title': 'Kình ngư đầu tiên của Việt Nam giành 2 HCV ở giải châu Á',
 'sapo': '(Dân trí) - Vận động viên (VĐV) bơi Nguyễn Huy Hoàng đã đi vào lịch sử khi trở thành kình ngư đầu tiên của Việt Nam giành 2 Huy chương vàng (HCV) ở giải châu Á khi về nhất nội dung 800m tự do nam vào tối 1/10.',
 'content': 'Kình ngư số một Việt Nam Nguyễn Huy Hoàng tiếp tục tỏa sáng rực rỡ tại giải vô địchthể thaodưới nước châu Á 2025. Tối nay (1/10), anh đã giành thêm HCV nội dung 800m tự do nam với thành tích 7 phút 57 giây 58. Huy Hoàng giành Huy chương vàng ở nội dung bơi 800m tại giải vô địch thể thao dưới nước châu Á 2025 (Ảnh chụp màn hình). Ở phần thi này, Huy Hoàng phải đối đầu với nhiều đối thủ mạnh như Rawat Kushagra (Ấn Độ), Khiew Hoe Yean (Malaysia), Xu Haibo (Trung Quốc), Sibirtsev Ilya (Uzbekistan) hay Tanaka Shun (Nhật Bản). Tuy nhiên, kình ngư quê Quảng Bình nhanh chóng

# Áp dụng 2 hàm trên 1 batch test

In [19]:
test_df = all_articles_df[:5]
test_df

,category,url
0,Kinh doanh,https://dantri.com.vn/kinh-doanh/gia-han-nhan-...
1,Kinh doanh,https://dantri.com.vn/kinh-doanh/gia-vang-tien...
2,Kinh doanh,https://dantri.com.vn/kinh-doanh/visa-k-trung-...
3,Kinh doanh,https://dantri.com.vn/kinh-doanh/hoang-huong-b...
4,Kinh doanh,https://dantri.com.vn/kinh-doanh/hoang-huong-b...


In [20]:
all_test_data = []

# test là dataframe chứa tất cả các bài báo
for _, row in test_df.iterrows():
    print(f'Category: {row["category"]} - URL: {row["url"]}')
    data = get_article_detail(row["url"])      
    data_reactions = get_reactions(data, row["url"])      
    all_test_data.append(data_reactions)                       

Category: Kinh doanh - URL: https://dantri.com.vn/kinh-doanh/gia-han-nhan-ho-so-vong-1-vietnam-esg-awards-2025-20251004095137197.htm
Category: Kinh doanh - URL: https://dantri.com.vn/kinh-doanh/gia-vang-tien-sat-moc-139-trieu-dongluong-20251004082107763.htm
Category: Kinh doanh - URL: https://dantri.com.vn/kinh-doanh/visa-k-trung-quoc-mo-rong-cua-thu-hut-nhan-tai-the-gioi-20251003220453495.htm
Lỗi khi lấy https://dantri.com.vn/kinh-doanh/visa-k-trung-quoc-mo-rong-cua-thu-hut-nhan-tai-the-gioi-20251003220453495.htm: 'NoneType' object has no attribute 'find_all'
Category: Kinh doanh - URL: https://dantri.com.vn/kinh-doanh/hoang-huong-bi-bat-chuyen-gia-chi-ro-thu-doan-tron-thue-qua-ho-kinh-doanh-20251003224737030.htm
Category: Kinh doanh - URL: https://dantri.com.vn/kinh-doanh/hoang-huong-bi-bat-he-lo-de-che-kinh-doanh-va-chieu-tro-quang-cao-no-20251003210628410.htm


In [21]:
all_test_data_df = pd.DataFrame(all_test_data)
all_test_data_df

,link,title,sapo,content,author_name,author_link,main_section,sub_section,post_date,total_reacts,detailed_reacts
0,https://dantri.com.vn/kinh-doanh/gia-han-nhan-...,Gia hạn nhận hồ sơ vòng 1 Vietnam ESG Awards 2025,(Dân trí) - Ban Tổ chức Diễn đàn ESG Việt Nam ...,VietnamESGAwards là điểm nhấn của Diễn đàn ESG...,Thảo Thu,https://dantri.com.vn/tac-gia/thao-thu-3019.htm,Kinh doanh,ESG - Phát triển bền vững,"Thứ bảy, 04/10/2025 - 10:02",0,"{'Bổ ích': 0, 'Xúc động': 0, 'Cảm hứng': 0, 'Đ..."
1,https://dantri.com.vn/kinh-doanh/gia-vang-tien...,Giá vàng tiến sát mốc 139 triệu đồng/lượng,"(Dân trí) - Đến 12 giờ ngày 4/10, cả vàng miến...","Tính đến 12 giờ ngày 4/10,giá vàngSJC trong nư...",Tri Túc,https://dantri.com.vn/tac-gia/tri-tuc-3617.htm,Kinh doanh,Tài chính,"Thứ bảy, 04/10/2025 - 08:26",2,"{'Bổ ích': 1, 'Xúc động': 1, 'Cảm hứng': 0, 'Đ..."
2,https://dantri.com.vn/kinh-doanh/visa-k-trung-...,None,None,None,None,None,None,None,None,0,{}
3,https://dantri.com.vn/kinh-doanh/hoang-huong-b...,Hoàng Hường bị bắt: Chuyên gia chỉ rõ thủ đoạn...,"(Dân trí) - Theo chuyên gia, một thủ đoạn trốn...","Trao đổi với phóng viênDân trí, chuyên gia thu...",Mỹ Tâm,https://dantri.com.vn/tac-gia/my-tam-3178.htm,Kinh doanh,Doanh nghiệp,"Thứ bảy, 04/10/2025 - 07:17",26,"{'Bổ ích': 15, 'Xúc động': 6, 'Cảm hứng': 1, '..."
4,https://dantri.com.vn/kinh-doanh/hoang-huong-b...,Hoàng Hường bị bắt: Hé lộ đế chế kinh doanh và...,(Dân trí) - Hoàng Hường đã xây dựng hệ sinh th...,Cơ quan Cảnh sát điều tra Bộ Công an đã khởi t...,Minh Huyền,https://dantri.com.vn/tac-gia/minh-huyen-3439.htm,Kinh doanh,Doanh nghiệp,"Thứ sáu, 03/10/2025 - 21:40",53,"{'Bổ ích': 23, 'Xúc động': 15, 'Cảm hứng': 13,..."


# Áp dụng trên toàn bộ các bài báo

In [22]:
# do phần lấy reactions chạy khá lâu nên em tạm bỏ qua bước này
# mỗi bài báo em cần load khoảng 30s để lấy reactions

all_data = []

for _, row in all_articles_df.iterrows():
    print(f'Category: {row["category"]} - URL: {row["url"]}')
    data = get_article_detail(row["url"])
    #data_reactions = get_reactions(data, row["url"])          
    data["total_reacts"] = 0
    data["detailed_reacts"] = None   
    all_data.append(data)

Category: Kinh doanh - URL: https://dantri.com.vn/kinh-doanh/gia-han-nhan-ho-so-vong-1-vietnam-esg-awards-2025-20251004095137197.htm
Category: Kinh doanh - URL: https://dantri.com.vn/kinh-doanh/gia-vang-tien-sat-moc-139-trieu-dongluong-20251004082107763.htm
Category: Kinh doanh - URL: https://dantri.com.vn/kinh-doanh/visa-k-trung-quoc-mo-rong-cua-thu-hut-nhan-tai-the-gioi-20251003220453495.htm
Lỗi khi lấy https://dantri.com.vn/kinh-doanh/visa-k-trung-quoc-mo-rong-cua-thu-hut-nhan-tai-the-gioi-20251003220453495.htm: 'NoneType' object has no attribute 'find_all'
Category: Kinh doanh - URL: https://dantri.com.vn/kinh-doanh/hoang-huong-bi-bat-chuyen-gia-chi-ro-thu-doan-tron-thue-qua-ho-kinh-doanh-20251003224737030.htm
Category: Kinh doanh - URL: https://dantri.com.vn/kinh-doanh/hoang-huong-bi-bat-he-lo-de-che-kinh-doanh-va-chieu-tro-quang-cao-no-20251003210628410.htm
Category: Kinh doanh - URL: https://dantri.com.vn/kinh-doanh/tao-ra-51-gdp-khoi-tu-nhan-khat-khao-duoc-trao-quyen-duoc-cong-

In [23]:
all_data_df = pd.DataFrame(all_data)
all_data_df

,link,title,sapo,content,author_name,author_link,main_section,sub_section,post_date,total_reacts,detailed_reacts
0,https://dantri.com.vn/kinh-doanh/gia-han-nhan-...,Gia hạn nhận hồ sơ vòng 1 Vietnam ESG Awards 2025,(Dân trí) - Ban Tổ chức Diễn đàn ESG Việt Nam ...,VietnamESGAwards là điểm nhấn của Diễn đàn ESG...,Thảo Thu,https://dantri.com.vn/tac-gia/thao-thu-3019.htm,Kinh doanh,ESG - Phát triển bền vững,"Thứ bảy, 04/10/2025 - 10:02",0,None
1,https://dantri.com.vn/kinh-doanh/gia-vang-tien...,Giá vàng tiến sát mốc 139 triệu đồng/lượng,"(Dân trí) - Đến 12 giờ ngày 4/10, cả vàng miến...","Tính đến 12 giờ ngày 4/10,giá vàngSJC trong nư...",Tri Túc,https://dantri.com.vn/tac-gia/tri-tuc-3617.htm,Kinh doanh,Tài chính,"Thứ bảy, 04/10/2025 - 08:26",0,None
2,https://dantri.com.vn/kinh-doanh/visa-k-trung-...,None,None,None,None,None,None,None,None,0,None
3,https://dantri.com.vn/kinh-doanh/hoang-huong-b...,Hoàng Hường bị bắt: Chuyên gia chỉ rõ thủ đoạn...,"(Dân trí) - Theo chuyên gia, một thủ đoạn trốn...","Trao đổi với phóng viênDân trí, chuyên gia thu...",Mỹ Tâm,https://dantri.com.vn/tac-gia/my-tam-3178.htm,Kinh doanh,Doanh nghiệp,"Thứ bảy, 04/10/2025 - 07:17",0,None
4,https://dantri.com.vn/kinh-doanh/hoang-huong-b...,Hoàng Hường bị bắt: Hé lộ đế chế kinh doanh và...,(Dân trí) - Hoàng Hường đã xây dựng hệ sinh th...,Cơ quan Cảnh sát điều tra Bộ Công an đã khởi t...,Minh Huyền,https://dantri.com.vn/tac-gia/minh-huyen-3439.htm,Kinh doanh,Doanh nghiệp,"Thứ sáu, 03/10/2025 - 21:40",0,None
...,...,...,...,...,...,...,...,...,...,...,...
6453,https://dantri.com.vn/tinh-yeu-gioi-tinh/lan-d...,"Lần đầu ra mắt nhà bạn trai, tôi ngay lập tức ...","(Dân trí) - Tôi hoảng hồn nhận ra, mình đã gây...",Tôi đã suy nghĩ khá lâu trước khi quyết định g...,Như Ý Cát Tường,https://dantri.com.vn/tac-gia/nhu-y-cat-tuong-...,Tình yêu - Giới tính,Chuyện của tôi,"Thứ tư, 09/07/2025 - 16:25",0,None
6454,https://dantri.com.vn/tinh-yeu-gioi-tinh/moi-l...,"Mỗi lần hẹn hò, bạn gái lại kiếm đủ loại lý do...","(Dân trí) - Nói chuyện này ra, tôi thấy có chú...","Tôi năm nay 26 tuổi, đã tốt nghiệp đại học và ...",Giang Hà,https://dantri.com.vn/tac-gia/giang-ha-3269.htm,Tình yêu - Giới tính,Chuyện của tôi,"Thứ tư, 09/07/2025 - 14:03",0,None
6455,https://dantri.com.vn/tinh-yeu-gioi-tinh/nho-m...,"Nhờ một tiếng gọi ""bố"" của con gái, cuộc đời t...",(Dân trí) - Lần đầu tiên nghe con cất tiếng gọ...,"Tôi từng nghĩ, sau một cuộc hôn nhân đổ vỡ, mì...",Chi Phạm,https://dantri.com.vn/tac-gia/chi-pham-3296.htm,Tình yêu - Giới tính,Chuyện của tôi,"Thứ tư, 09/07/2025 - 10:55",0,None
6456,https://dantri.com.vn/tinh-yeu-gioi-tinh/toi-c...,"Tôi cưới vợ trẻ kém 30 tuổi, con trai phản đối...","(Dân trí) - Vài hôm trước, tôi bắt gặp con đứn...","Tôi năm nay 58 tuổi, có một cậu con trai 28 tu...",Giang Hà,https://dantri.com.vn/tac-gia/giang-ha-3269.htm,Tình yêu - Giới tính,Chuyện của tôi,"Thứ tư, 09/07/2025 - 08:49",0,None


In [29]:
len(all_data_df)

6458

In [30]:
all_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6458 entries, 0 to 6457
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   link             6458 non-null   object
 1   title            5917 non-null   object
 2   sapo             5917 non-null   object
 3   content          5917 non-null   object
 4   author_name      5917 non-null   object
 5   author_link      5917 non-null   object
 6   main_section     5917 non-null   object
 7   sub_section      3642 non-null   object
 8   post_date        5917 non-null   object
 9   total_reacts     6458 non-null   int64 
 10  detailed_reacts  0 non-null      object
dtypes: int64(1), object(10)
memory usage: 555.1+ KB


In [24]:
# Save to CSV
all_data_df.to_csv("dantri_articles.csv", index=False)